## Project 1: Lexical Normalisation of Twitter Data

---
### Overview

The goal of this Project is to assess the performance of some spelling correction methods on the problem
of tweet normalisation, and to express the knowledge that you have gained in a technical report. This
aims to reinforce concepts in approximate matching and evaluation, and to strengthen your skills in data
analysis and problem solving.

### Deliverables

1. One or more programs, implemented in one or more programming languages, which must:
    - Determine the best match(es) for a token, with respect to a reference collection (dictionary)
    - Process the data input file(s), to determine the best match for each token
    - Evaluate the matches, with respect to the truly intended words, using one or more evaluation
       metrics
2. A README thatbrieflydetails how your program(s) work(s). You may use any external re-
    sources for your program(s) that you wish: you must indicate these, and where you obtained
    them, in your README. The program(s) and README are required submission elements, but
    will not typically be directly assessed.
3. A technical report, of 1000–1600 words, which must:
    - Give a short description of the problem and data set
    - Brieflysummarise some relevant literature
    - Briefly explain the approximate matching technique(s), and how it is (they are) used
    - Present the results, in terms of the evaluation metric(s) and illustrative examples
    - Contextualise the system’s behaviour, based on the (admittedly incomplete) understanding
       from the subject materials
    - Clearlydemonstrate some knowledge about the problem

### Assessment Criteria

Method: (20% of the marks available)
You will attempt a representative sample of approximate matching techniques, which is adequate for
deriving some knowledge about the problem of tweet normalisation. You will evaluate your method(s)
formally.

Critical Analysis: (50% of the marks available)
You will explain the practical behaviour of your systems, referring to the theoretical behaviour where
appropriate. You will support your observations with evidence, in terms of illustrative examples and
evaluation metrics. You will derive some knowledge about the problem of tweet normalisation.

Report Quality: (30% of the marks available)
You will produce a formal report, which is commensurate in style and structure with a (short) research
paper. You must express your ideas clearly and concisely, and remain within the word limit (1000-
words). You will include a short summary of related research.

We will post a marking rubric to indicate what we will be looking for in each of these categories
when marking.

---
### Read Me

The data package is in 6 files:

      README.txt: this file, which describes the data and its format.

      {labelled,unlabelled}-tweets.txt: A list of tweets, one per line.  Each tweet
      is comprised of a space-delimited list of tokens. The tweets have been
      case-folded, where any upper-case (English) characters have been converted to
      their lower-case equivalent.

      {labelled,unlabelled}-tokens.txt: A list of tokens, one per line. The tokens
      are drawn from the tweets above, excepts that tokens not containing at least
      one (English) alphabetical character - like "." or "!!" - have been excluded.
      The order of the tokens corresponds to the tweets; no indication of the tweet
      boundary is given, but this can be recovered from the tweets files as
      necessary.

  The format of these files follows.

  For the labelled tweets:
     
      Token TAB Code TAB Canonical_Form
     
  Where "Token" is drawn from the tweet text (suitably down-cased),
 
 "Canonical_Form" is the normalised version of the token, and "Code" can take
  one of three values: 
    
    IV - "in vocabulary", such that the form from the tweet was found in the
    dictionary, and is consequently not a candidate for normalisation. (There
    are some exceptions to this, which can be construed as mistakes in the
    data.)
    
    OOV - "out of vocabulary", such that the form of the token from the tweet
    was not found in the dictionary, and thus the token was a candidate for
    normalisation. In some cases, the canonical form is homologous (equivalent)
    to the un-normalised form. In other cases, they are different --- these are
    the "spelling mistakes" that need to be "corrected", in light of the
    Project requirements.
    
    NO - "not a normalisation candidate", such that the token was not
    considered in the normalisation process.
  
  For example:
    
        new     IV      new
    
  "new" is in the dictionary, so its canonical form is equivalent to its form 
  from the tweet.
    
        pix     OOV     pictures
    
  "pix" is not in the dictionary, and its canonical form is "pictures".
    
        #heatchuuuuu    NO      #heatchuuuuu
    
  "#heatchuuuuu" is not in the dictionary, but it is a Twitter hashtag, and 
  consequently was not regarded as a candidate for normalisation.

  For the unlabelled tweets:
   
       token TAB ???
   
  Where "token" is drawn from the tweet text (suitably down-cased). The "code"
  and "canonical form" are not given. If you wish to test your system on this
  dataset, you can submit to Kaggle, as explained on the LMS.

  For example:
  
        rt      ???
  
  "rt" may or may not be in the dictionary, and there may or may not be a
  canonical form for this token.
  
---

### Preamble

In [91]:
import editdistance
import jellyfish as jf
from collections import Counter
import re
from autocorrect import spell
import nltk
import py_stringmatching as sm

porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
snowball = nltk.SnowballStemmer("english", ignore_stopwords=True)
wnl = nltk.WordNetLemmatizer()

### Step 1 – Levenshtein Distance 

Once a candidate has been identified for normalisation, firstly, 
edit distance (Levenshtein distance) technique is applied to find matches from (dict.txt) 
which are within 2 (inclusive) edit distance of the query. The results are stored in an array. 
We refer to this set as the “First Set of Matches based on Edit Distance” 
since they contain approximate matches based on their textual similarity to the query.

In [92]:
def levenshtein_distance(string, dictionary):   
    distance_dict = [x for x in dictionary if editdistance.eval(string, x) <= 2]
    return distance_dict

### Step 2 – Phonetic Matching

Soundex is used to further analyse and phonetically match words gathered in the first set, based on their Levenshtein distance to the query. The words in the array are filtered based on their phonetic similarity to the query using soundex, refined soundex and metaphone, etc.

In [93]:
from pyphonetics import RefinedSoundex
rs = RefinedSoundex()
def soundex_filter(string, dictionary):
    
    filter_once = [x for x in dictionary if rs.distance(string, x, metric='levenshtein') == 0] 
#     filter_twice = [x for x in filter_once if jf.metaphone(x) == jf.metaphone(string)] 
    
    return filter_once

### Step 3 – [Peter Norvig’s Algorithm](http://norvig.com/spell-correct.html)

Algorithm generates all possible terms with an edit distance of less than or equal to 2 (which includes deletes, transposes, replaces, and inserts) from the query term and searches them in the dictionary (big.txt).

In [94]:
# source: http://norvig.com/spell-correct.html
# author: Peter Norvig

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Step 4 – More Spell Checking 

Using Peter Norvig’s Algorithm, python autocorrect package and other stemmers from nltk library to perform a cross-evaluation of the spell chekcer results.

In [95]:
def spellChecking(candidate):
    wordlist = []
    wordlist.append(correction(candidate))
    wordlist.append(spell(candidate))
    #wordlist.append(porter.stem(candidate))
    #wordlist.append(lancaster.stem(candidate))
    wordlist.append(snowball.stem(candidate))
    #wordlist.append(wnl.lemmatize(candidate))
    
    return wordlist

## Normalisation of Unlabelled Tokens

### Step 5 - [CMU Twitter POS Tagger](http://www.cs.cmu.edu/~ark/TweetNLP/)

Using CMU twitter part-of-speench tagger to preprocess the unlabelled tokens and further narrow down potential candidates for normalisation

In [96]:
tags = open('POS.txt', 'r', encoding='latin-1')
tag = {}
for line in tags:
    words = line.split()
    tag[words[0]] = (words[1],words[2])

### Step 6 - Using various lexical normalisation dictionaries

In [97]:
def most_common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

f =  open('labelled-tokens.txt', encoding='latin-1')

labelled_tokens = {}
for line in f:
    words = line.split()
    if words[0] in labelled_tokens:
        labelled_tokens[words[0]].append(words[2])
    else:
        labelled_tokens[words[0]] = [words[2]]

for key, value in labelled_tokens.items():
    labelled_tokens[key] = most_common(labelled_tokens[key])
    
f.close()

In [9]:
f3 = open('Test_Set_3802_Pairs.txt')

dict3 = {}

for line in f3:
    words = line.split()
    if int(words[0]) >= 4:  
        dict3[words[1]] = words[3]

f3.close()

In [10]:
f3 = open('Test_Set_3802_Pairs.txt')

dict3_complete = {}

for line in f3:
    words = line.split()
    just_words = [x for x in words if x != "|"]
    dict3_complete[just_words[1]] = just_words[2:]

f3.close()

In [105]:
f =  open('labelled-tokens.txt', encoding='latin-1')
f1 = open('dict.txt')
f2 = open('emnlp_dict.txt')

# dictionary look up for IV words
dict1 = []
for line in f1:
    words = line.split()
    dict1.append(words[0])
    
    
keys=[]
values=[]
for line in f2:
    words = line.split()
    keys.append(words[0])
    if len(words) == 2:
        values.append(words[1])
    else:
        values.append(" ".join(words[1:]))
    
dict2 = dict(zip(keys,values))


OOV_tokens=[]
correct=[]
for line in f:
    words = line.split()
    if words[1] != 'IV' and words[1] != 'NO':
        OOV_tokens.append(words[0])
        correct.append(words[2])
    

f.close()
f1.close()
f2.close()

In [99]:
f4 = open('w5_.txt')

w5 = []

for line in f4:
    item = line.split()
    w5.append(item[1]+item[2]+item[3]+item[4]+item[5])
    
f4.close()


In [ ]:
unlabelled =  open('unlabelled-tokens.txt', 'r', encoding='latin-1')
# list of unlabelled tokens
unlabelled_tokens = []
for line in unlabelled:
    words = line.split()
    unlabelled_tokens.append(words[0])

normalisation = []

for index, item in enumerate(unlabelled_tokens):
    if re.match('^.*[:/_&%@#~<=>`",;\'\-\.\!\$\s\?\+\*\^\|\[\]\(\)\{\}]+.*$', item):
        normalisation.append(item)
        
    elif item in dict3:
        normalisation.append(dict3[item])
        
    elif item in labelled_tokens:
        normalisation.append(labelled_tokens[item])
        
    elif item in dict2:
        normalisation.append(dict2[item])
        
    elif item in dict1:
        normalisation.append(item)
        
    else:
        wordlist = []
        if item in dict3_complete:
            if len(dict3_complete[item]) == 1:
                normalisation.append(item)
                continue
            else:
                wordlist.extend(dict3_complete[item])
        
        peter = correction(item)
        wordlist.extend(soundex_filter(item, levenshtein_distance(item, dict1)))
        
        if peter in wordlist:
            normalisation.append(peter)
            continue
        else:
            wordlist.append(peter)
            wordlist.extend(spellChecking(item))
            
            if unlabelled_tokens[index-1]  in dict1 and unlabelled_tokens[index+1]  in dict1:
                freq = {}
                for word in wordlist:
                    r = re.compile(".*%s%s%s.*" % (unlabelled_tokens[index-1], word, unlabelled_tokens[index+1]))
                    newlist = filter(r.match, w5)
                    freq[word] = len(list(newlist)) 
                normalisation.append(max(freq.keys(), key=(lambda k: freq[k])))
                continue
            else:
                fdist = nltk.FreqDist(wordlist) 
                normalisation.append(fdist.max())
                continue
        
normalisation

### Best Accuracy Score

In [106]:
unlabelled =  open('unlabelled-tokens.txt', 'r', encoding='latin-1')
unlabelled_tokens = []
for line in unlabelled:
    words = line.split()
    unlabelled_tokens.append(words[0])

normalisation = []

for index, item in enumerate(unlabelled_tokens):
    if re.match('^.*[:/_&%@#~<=>`",;\.\!\$\s\?\+\*\^\|\[\]\(\)\{\}]+.*$', item):
        normalisation.append(item)
        
    elif item in dict3:
        normalisation.append(dict3[item])
        
    elif item in labelled_tokens:
        normalisation.append(labelled_tokens[item])
        
    elif item in dict2:
        normalisation.append(dict2[item])
        
    elif item in dict1:
        normalisation.append(item)
        
    elif tag[item][0] in ["N","!","^","Z","E","~","$","A","V","G","R","L"]:
        normalisation.append(item)
    
    else:
        wordlist = spellChecking(item)
        
        if item in dict3_complete:
            wordlist.extend(dict3_complete[item])
        
        wordlist.extend(soundex_filter(item, levenshtein_distance(item, dict1)))
                
        fdist = nltk.FreqDist(wordlist) 
        
        normalisation.append(fdist.max())
        
normalisation

['rt',
 '@teddyferrari1',
 'ah',
 '@datzmenoni',
 'why',
 'sub',
 'ozil',
 '@lexzydoo_ab',
 'opolo',
 'eyes',
 'you',
 'no',
 'fit',
 'open',
 'eyes',
 'you',
 'have',
 'a',
 'very',
 'sexy',
 'header',
 '@jaibrooks1',
 'roar',
 'i',
 'miss',
 'you',
 'my',
 'bie',
 'where',
 'you',
 'wanna',
 'out',
 'with',
 'me',
 'have',
 'a',
 'wonderful',
 'day',
 'like',
 'a',
 'swan',
 'day',
 'haha',
 'cantik',
 'rt',
 '@historyinpics',
 'julie',
 'christie',
 'photograph',
 'by',
 'richard',
 'avedon',
 'http://t.co/gscav8o0ik',
 'rt',
 '@fvckxhemmings',
 'did',
 'calum',
 'slip',
 'omfg',
 '@daviddarkshade',
 'and',
 'you',
 "didn't",
 'make',
 'this',
 'brother',
 'my',
 'friend',
 'jack',
 'storm',
 'did',
 '@xoxoxyareli',
 '@billmillerprobz',
 'pshh',
 'i',
 'went',
 'and',
 'got',
 'mine',
 'done',
 'regardless',
 "i'm",
 'sick',
 'of',
 'not',
 'being',
 'able',
 'to',
 'be',
 'a',
 'girl',
 'haha',
 '@tindency',
 'syempre',
 'in',
 'the',
 'right',
 'age',
 'haha',
 '@khalilbrown_',
 '

### Write to txt for submission

In [107]:
# write to file 
kaggle = open('kaggle.txt','w')
kaggle.write('Token,Prediction' + '\n')
i=1
for line in normalisation:
    kaggle.write(str(i) + ',' + line + '\n')
    i=i+1
    
kaggle.close()

### Evaluation metrics
---
** Accuracy **

In [ ]:
def accuracy(actual, prediction):
    i = 0
    for x,y in zip(actual, prediction):
        if x == y:
            i = i + 1
    return i/len(actual)  

In [104]:
fdist = nltk.FreqDist([x[1] for x in dontknow]) 
import operator
sorted_x = sorted(fdist.items(), key=operator.itemgetter(1),reverse=True)
sorted_x

[('exo', 31),
 ('zayn', 23),
 ('cr', 22),
 ('vs', 19),
 ('tbh', 16),
 ('ive', 16),
 ('snapchat', 16),
 ('idc', 15),
 ('playing', 15),
 ('selfie', 15),
 ('luhan', 13),
 ('sehun', 13),
 ('croke', 13),
 ('chanyeol', 12),
 ('desain', 12),
 ('followers', 12),
 ('ng', 11),
 ('jfb', 11),
 ('suho', 11),
 ('lmaoo', 11),
 ('ukip', 10),
 ('lebron', 10),
 ('shenanigans', 10),
 ('harrystyles', 10),
 ('kamar', 10),
 ('ang', 10),
 ('psg', 9),
 ('ilysm', 9),
 ('mixtape', 9),
 ('tf', 9),
 ('playlist', 9),
 ('rn', 9),
 ('kik', 9),
 ('selfies', 8),
 ('ideas', 8),
 ('kardashian', 8),
 ('malaysia', 8),
 ('boko', 8),
 ('haram', 8),
 ('instagram', 8),
 ('5sos', 8),
 ('shinee', 8),
 ('xo', 7),
 ('miley', 7),
 ('hemmings', 7),
 ('anak', 7),
 ('ke', 7),
 ('ronaldo', 7),
 ('modi', 7),
 ('ily', 7),
 ('3rd', 7),
 ('started', 7),
 ('iggy', 7),
 ('zouis', 7),
 ('wanted', 7),
 ('calum', 6),
 ('usa', 6),
 ('voted', 6),
 ('hashtag', 6),
 ('yg', 6),
 ('hq', 6),
 ('cewek', 6),
 ('avi', 6),
 ('rey', 6),
 ('seoul', 6),
 ('

In [86]:
a = []
for key, item in tag.items():
    a.append(item[0])
    
fdist = nltk.FreqDist(a) 
fdist

FreqDist({'!': 2218,
          '#': 432,
          '$': 70,
          '&': 15,
          ')': 2,
          ',': 4,
          '@': 2024,
          'A': 592,
          'D': 24,
          'E': 28,
          'G': 190,
          'L': 96,
          'N': 2593,
          'O': 64,
          'P': 67,
          'R': 179,
          'S': 16,
          'T': 6,
          'U': 940,
          'V': 1197,
          'Y': 1,
          'Z': 22,
          '^': 1916,
          'leaveayoutubecomment': 1,
          'northside': 1,
          '~': 2})

In [85]:
sorted(fdist,key=fdist.get, reverse=True)

['N',
 '!',
 '@',
 '^',
 'V',
 'U',
 'A',
 '#',
 'G',
 'R',
 'L',
 '$',
 'P',
 'O',
 'E',
 'D',
 'Z',
 'S',
 '&',
 'T',
 ',',
 '~',
 ')',
 'Y',
 'leaveayoutubecomment',
 'northside']

In [90]:
tag

{'rt': ('~', '0.4510'),
 '@teddyferrari1': ('@', '0.9989'),
 'ah': ('!', '0.9960'),
 '@datzmenoni': ('@', '0.9990'),
 'why': ('R', '0.8527'),
 'sub': ('N', '0.9591'),
 'ozil': ('^', '0.6393'),
 '@lexzydoo_ab': ('@', '0.9990'),
 'opolo': ('!', '0.6633'),
 'eyes': ('N', '0.9459'),
 'u': ('O', '0.9843'),
 'no': ('!', '0.9253'),
 'fit': ('V', '0.5119'),
 'open': ('V', '0.5185'),
 'have': ('V', '0.9703'),
 'a': ('D', '0.8948'),
 'very': ('R', '0.9132'),
 'sexy': ('A', '0.9845'),
 'header': ('N', '0.9654'),
 '@jaibrooks1': ('@', '0.9989'),
 'rawr': ('!', '0.9815'),
 'i': ('O', '0.9651'),
 'miss': ('V', '0.6716'),
 'my': ('D', '0.8420'),
 'bie': ('^', '0.8871'),
 'where': ('R', '0.9493'),
 'wanna': ('V', '0.9079'),
 'out': ('T', '0.6015'),
 'wif': ('!', '0.4851'),
 'me': ('O', '0.9918'),
 'wonderful': ('A', '0.9407'),
 'day': ('N', '0.9236'),
 'like': ('P', '0.7012'),
 'swan': ('N', '0.8017'),
 'haha': ('!', '0.9993'),
 'cantik': ('!', '0.4007'),
 '@historyinpics': ('@', '0.9990'),
 'julie': 